In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import sqlite3
import pickle
pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from functools import reduce 

In [2]:
df_data = pd.read_pickle('data_merged1.pkl')

In [4]:
df_data.drop(['index_x', 'index_y'], axis=1, inplace=True)

,LSOA code,year,LSOA population,District name,district code,Geo code police,Force Name,population,headcount,Month,LSOA name,amount_of_crimes,month
0,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-01,County Durham 001A,3,1
1,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-02,County Durham 001A,4,2
2,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-03,County Durham 001A,11,3
3,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-04,County Durham 001A,4,4
4,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-05,County Durham 001A,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112306,E01018744,2019,1615.0,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-08,Cheshire West and Chester 037D,10,8
1112307,E01018744,2019,1615.0,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-09,Cheshire West and Chester 037D,4,9
1112308,E01018744,2019,1615.0,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-10,Cheshire West and Chester 037D,20,10
1112309,E01018744,2019,1615.0,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-11,Cheshire West and Chester 037D,8,11


In [17]:
df = pd.merge(ukdpg, df_data, on=['year', 'district code'])

In [22]:
df_IMD = pd.read_csv("IMD_2015_LSOA_score.csv", skiprows = 7)

df_IMD = df_IMD[["Reference area", "2015"]]
df_IMD = df_IMD.rename(columns={"Reference area":"LSOA name", "2015" : "IMD score"})



In [25]:
data = pd.merge(df_data, df_IMD,  on=['LSOA name']).dropna()
data.reset_index(inplace=True)

In [42]:
data['prev_month_crimes'] = data.groupby(['LSOA name', 'year'])['amount_of_crimes'].shift(1)


In [44]:
data.to_pickle("final_data.pkl")  

In [53]:
test = data[data['year'] == 2019]
train = data[data['year']<2019]

In [54]:
test.drop(['LSOA code', 'Geo code police', 'district code', 'Month', 'index'], axis=1, inplace=True)
train.drop(['LSOA code', 'Geo code police', 'district code', 'Month', 'index'], axis=1, inplace=True)


C:\Users\blaze\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [57]:
lab = LabelEncoder()

labels_lsoa = data['LSOA name'].unique()
lab.fit(labels_lsoa)
train[['LSOA name']] = train[['LSOA name']].apply(lab.fit_transform)
test[['LSOA name']] = test[['LSOA name']].apply(lab.fit_transform)

C:\Users\blaze\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [58]:
labels_dist = data['District name'].unique()
lab.fit(labels_dist)
train[['District name']] = train[['District name']].apply(lab.fit_transform)
test[['District name']] = test[['District name']].apply(lab.fit_transform)

In [59]:
labels_force = data['Force Name'].unique()
lab.fit(labels_force)
train[['Force Name']] = train[['Force Name']].apply(lab.fit_transform)
test[['Force Name']] = test[['Force Name']].apply(lab.fit_transform)

In [60]:
train

,year,LSOA population,District name,Force Name,population,headcount,LSOA name,amount_of_crimes,month,IMD score,prev_month_crimes
0,2012,1996.0,35,5,514261,1383,192,4,1,6.214,NaN
1,2012,1996.0,35,5,514261,1383,192,4,2,6.214,4.0
2,2012,1996.0,35,5,514261,1383,192,11,3,6.214,4.0
3,2012,1996.0,35,5,514261,1383,192,5,4,6.214,11.0
4,2012,1996.0,35,5,514261,1383,192,5,5,6.214,5.0
...,...,...,...,...,...,...,...,...,...,...,...
801692,2018,1123.0,142,1,209547,2050,191,21,8,18.421,9.0
801693,2018,1123.0,142,1,209547,2050,191,16,9,18.421,21.0
801694,2018,1123.0,142,1,209547,2050,191,12,10,18.421,16.0
801695,2018,1123.0,142,1,209547,2050,191,9,11,18.421,12.0


In [63]:
X_train = train[['year', 'LSOA population', 'District name', 'Force Name', 'population', 'headcount', 'LSOA name', 'month', 'IMD score']]
y_train = train['amount_of_crimes']

X_test = test[['year', 'LSOA population', 'District name', 'Force Name', 'population', 'headcount', 'LSOA name', 'month', 'IMD score']]
y_test = test['amount_of_crimes']

In [64]:
cl = RandomForestRegressor(n_estimators = 10, random_state = 42)
cl.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [69]:
def evaluate(model):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f'The mae of the model is {mae}.')
    print(f'The mse of the model is {mse}, using weighted average.')
    print(f'The rmse of the model is {rmse}, using weighted average.')
    print(f'The r2 of the model is {r2}, using weighted average.')
    
    return rmse

In [70]:
evaluate(cl)

The mae of the model is 7.886373016509054.
The mse of the model is 251.95618959368485, using weighted average.
The rmse of the model is 15.873127908313624, using weighted average.
The r2 of the model is 0.6176025017574074, using weighted average.


15.873127908313624

In [72]:
X_train

,year,LSOA population,District name,Force Name,population,headcount,LSOA name,month,IMD score
0,2012,1996.0,35,5,514261,1383,192,1,6.214
1,2012,1996.0,35,5,514261,1383,192,2,6.214
2,2012,1996.0,35,5,514261,1383,192,3,6.214
3,2012,1996.0,35,5,514261,1383,192,4,6.214
4,2012,1996.0,35,5,514261,1383,192,5,6.214
...,...,...,...,...,...,...,...,...,...
801692,2018,1123.0,142,1,209547,2050,191,8,18.421
801693,2018,1123.0,142,1,209547,2050,191,9,18.421
801694,2018,1123.0,142,1,209547,2050,191,10,18.421
801695,2018,1123.0,142,1,209547,2050,191,11,18.421


In [74]:
cl.feature_importances_

array([1.77401084e-02, 1.59335741e-01, 4.09059731e-04, 2.73855139e-02,
       9.33081701e-04, 3.01452988e-02, 1.34281333e-01, 4.36719288e-02,
       5.86097934e-01])